<a href="https://colab.research.google.com/github/kridtapon/WFO-Momentum-Matrix/blob/main/WFO_Momentum_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.8 MB/s eta 0:00:00


In [34]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """
    Calculate MACD, Signal Line, and Histogram.
    """
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

# Function to calculate Ichimoku Kijun-Sen (Baseline)
def calculate_kijun_sen(df, period=26):
    """
    Calculate Ichimoku Kijun-Sen (Baseline).
    """
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()
    kijun_sen = (high_rolling + low_rolling) / 2
    return kijun_sen

# Define the stock symbol and time period
symbol = 'NRG' # NRG
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with MACD and Kijun-Sen
def walk_forward_optimization_macd_kijun(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    short_window_range = range(5, 15, 2)
    long_window_range = range(15, 31, 2)
    signal_window_range = range(5, 20, 2)
    kijun_sen_period_range = range(5, 20)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of MACD and Kijun-Sen parameters
        for params in itertools.product(short_window_range, long_window_range, signal_window_range, kijun_sen_period_range):
            short_window, long_window, signal_window, kijun_sen_period = params

            # Calculate MACD and Kijun-Sen on the training data
            train_data['MACD'], train_data['Signal_Line'] = calculate_macd(train_data, short_window, long_window, signal_window)
            train_data['Kijun_Sen'] = calculate_kijun_sen(train_data, kijun_sen_period)

            # Generate entry and exit signals
            entries = (train_data['MACD'] > train_data['Signal_Line']) & (train_data['Close'] > train_data['Kijun_Sen'])
            exits = (train_data['MACD'] < train_data['Signal_Line']) & (train_data['Close'] < train_data['Kijun_Sen'])

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (short_window, long_window, signal_window, kijun_sen_period)

        # Test with the best parameters on the test data
        test_data['MACD'], test_data['Signal_Line'] = calculate_macd(test_data, best_params[0], best_params[1], best_params[2])
        test_data['Kijun_Sen'] = calculate_kijun_sen(test_data, best_params[3])

        entries = (test_data['MACD'] > test_data['Signal_Line']) & (test_data['Close'] > test_data['Kijun_Sen'])
        exits = (test_data['MACD'] < test_data['Signal_Line']) & (test_data['Close'] < test_data['Kijun_Sen'])

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)


# Perform walk-forward optimization
results = walk_forward_optimization_macd_kijun(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply MACD and Kijun-Sen indicators
    yearly_data['MACD'], yearly_data['Signal_Line'] = calculate_macd(yearly_data, params[0], params[1], params[2])
    yearly_data['Kijun_Sen'] = calculate_kijun_sen(yearly_data, params[3])

    # Define entry/exit conditions
    entries = (yearly_data['MACD'] > yearly_data['Signal_Line']) & (yearly_data['Close'] > yearly_data['Kijun_Sen'])
    exits = (yearly_data['MACD'] < yearly_data['Signal_Line']) & (yearly_data['Close'] < yearly_data['Kijun_Sen'])

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-34-0001a3512ded>:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-34-0001a3512ded>:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-34-0001a3512ded>:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide


Walk-Forward Optimization Results:
   Year      Best_Params
0  2020  (13, 27, 19, 6)
1  2021  (5, 15, 17, 13)
2  2022  (5, 19, 13, 13)
3  2023   (5, 19, 13, 8)
4  2024   (5, 19, 13, 8)
5  2025   (5, 15, 11, 5)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           231079.349686
Total Return [%]                        131.07935
Benchmark Return [%]                   175.103187
Max Gross Exposure [%]                      100.0
Total Fees Paid                      15375.346674
Max Drawdown [%]                        19.786162
Max Drawdown Duration                       246.0
Total Trades                                   46
Total Closed Trades                            46
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                           

<ipython-input-34-0001a3512ded>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-34-0001a3512ded>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-34-0001a3512ded>:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-34-0

In [35]:
# Filter Test Years
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,275103.18711
Total Return [%],175.103187
Benchmark Return [%],175.103187
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],45.107354


Reverse Technical

In [33]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate MACD
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """
    Calculate MACD, Signal Line, and Histogram.
    """
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

# Function to calculate Ichimoku Kijun-Sen (Baseline)
def calculate_kijun_sen(df, period=26):
    """
    Calculate Ichimoku Kijun-Sen (Baseline).
    """
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()
    kijun_sen = (high_rolling + low_rolling) / 2
    return kijun_sen

# Define the stock symbol and time period
symbol = 'BTC-USD' #WMT
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with MACD and Kijun-Sen
def walk_forward_optimization_macd_kijun(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    short_window_range = range(5, 20, 2)
    long_window_range = range(20, 41, 2)
    signal_window_range = range(5, 20, 2)
    kijun_sen_period_range = range(5, 20)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of MACD and Kijun-Sen parameters
        for params in itertools.product(short_window_range, long_window_range, signal_window_range, kijun_sen_period_range):
            short_window, long_window, signal_window, kijun_sen_period = params

            # Calculate MACD and Kijun-Sen on the training data
            train_data['MACD'], train_data['Signal_Line'] = calculate_macd(train_data, short_window, long_window, signal_window)
            train_data['Kijun_Sen'] = calculate_kijun_sen(train_data, kijun_sen_period)

            # Generate entry and exit signals
            entries = (train_data['MACD'] > train_data['Signal_Line']) & (train_data['Close'] < train_data['Kijun_Sen'])
            exits = (train_data['MACD'] < train_data['Signal_Line']) & (train_data['Close'] > train_data['Kijun_Sen'])

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (short_window, long_window, signal_window, kijun_sen_period)

        # Test with the best parameters on the test data
        test_data['MACD'], test_data['Signal_Line'] = calculate_macd(test_data, best_params[0], best_params[1], best_params[2])
        test_data['Kijun_Sen'] = calculate_kijun_sen(test_data, best_params[3])

        entries = (test_data['MACD'] > test_data['Signal_Line']) & (test_data['Close'] < test_data['Kijun_Sen'])
        exits = (test_data['MACD'] < test_data['Signal_Line']) & (test_data['Close'] > test_data['Kijun_Sen'])

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)


# Perform walk-forward optimization
results = walk_forward_optimization_macd_kijun(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply MACD and Kijun-Sen indicators
    yearly_data['MACD'], yearly_data['Signal_Line'] = calculate_macd(yearly_data, params[0], params[1], params[2])
    yearly_data['Kijun_Sen'] = calculate_kijun_sen(yearly_data, params[3])

    # Define entry/exit conditions
    entries = (yearly_data['MACD'] > yearly_data['Signal_Line']) & (yearly_data['Close'] < yearly_data['Kijun_Sen'])
    exits = (yearly_data['MACD'] < yearly_data['Signal_Line']) & (yearly_data['Close'] > yearly_data['Kijun_Sen'])

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

[*********************100%***********************]  1 of 1 completed
<ipython-input-33-6531c9e71dd4>:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-6531c9e71dd4>:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-6531c9e71dd4>:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide


Walk-Forward Optimization Results:
   Year      Best_Params
0  2020   (5, 24, 7, 16)
1  2021   (7, 34, 17, 8)
2  2022    (5, 34, 5, 8)
3  2023    (5, 24, 7, 7)
4  2024  (17, 38, 19, 9)
5  2025    (5, 22, 5, 8)
Start                                2020-01-01 00:00:00
End                                  2024-12-31 00:00:00
Period                                1827 days 00:00:00
Start Value                                     100000.0
End Value                                   87456.207625
Total Return [%]                              -12.543792
Benchmark Return [%]                         1197.596406
Max Gross Exposure [%]                             100.0
Total Fees Paid                              3982.999384
Max Drawdown [%]                               72.626157
Max Drawdown Duration                 1447 days 00:00:00
Total Trades                                          30
Total Closed Trades                                   30
Total Open Trades                               

<ipython-input-33-6531c9e71dd4>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-6531c9e71dd4>:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-6531c9e71dd4>:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-33-6